In [1]:
from dask_gateway import Gateway
gateway = Gateway()

# make sure all the clusters are closed (This is expensive!)
clusters = gateway.list_clusters()
for cluster in clusters:
    gateway.stop_cluster(cluster.name)

options = gateway.cluster_options()

# # set the options programatically, or through their HTML repr
options.worker_memory = 16  # 10 GB of memory per worker.

# # Create a cluster with those options
cluster = gateway.new_cluster(options, shutdown_on_close=True)
cluster.adapt(minimum=6, maximum=40)
# cluster.scale(6)
client = cluster.get_client()
client

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.d0a0b574c94246cf81e935990a11db74/status,


In [3]:
cluster.adapt(minimum=6, maximum=40)

In [4]:
# load the ocean data
import fsspec
import xarray as xr
import xesmf as xe
import os
from intake import open_catalog
kwargs = dict(consolidated=True, use_cftime=True)
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/GFDL_CM2.6.yaml")
ds_ocean  = cat["GFDL_CM2_6_control_ocean_surface"].to_dask()
ds_flux  = cat["GFDL_CM2_6_control_ocean_boundary_flux"].to_dask()
ds_atmos = xr.open_zarr('gs://cmip6/GFDL_CM2_6/control/atmos_daily.zarr', **kwargs)
ds_oc_grid  = cat["GFDL_CM2_6_grid"].to_dask()
# cut to same time
all_dims = set(list(ds_ocean.dims)+list(ds_atmos.dims))
ds_ocean, ds_atmos = xr.align(
    ds_ocean,
    ds_atmos,
    join='inner',
    exclude=(di for di in all_dims if di !='time')
)

In [ ]:
# merge daily variables
ds = xr.merge([ds_ocean[]])

In [ ]:
calgo = "andreas"
zt = 10.0
zu = 10.0

sst = np.full(shape, 290.0, order=order)
t_zt = np.full(shape, 290.0, order=order)
hum_zt = np.full(shape, 0.001, order=order)
u_zu = np.full(shape, 1.0, order=order)
v_zu = np.full(shape, -1.0, order=order)
slp = np.full(shape, 101000.0, order=order)
niter = 15  # make optional
tic = time()
ql, qh, tau_x, tau_y, evap = aero.mod_aerobulk_wrapper.aerobulk_model_noskin(
    calgo, zt, zu, sst, t_zt, hum_zt, u_zu, v_zu, slp, niter
)

In [1]:
import aerobulk.mod_aerobulk_wrap as aero